In [1]:
from peratouch.data import Data, load_data
from peratouch.trainer import Trainer 
from peratouch.results import Results 
from peratouch.networks import CNN
from peratouch.config import path_five_users_main, path_five_users_first
import sklearn
import numpy as np

def run_n_presses(X, y, n_press, n_folds=5):
    """
    Runs entire routine of fitting CNN model to dataset (X, y)self.
    Performs Cross-Validation of n_folds on input dataset.
    Assumes data is in temporal order, and NOT shuffled.
    """

    D = Data(X, y)
    D.group_presses(n_press=n_press)
    D.shuffle()

    # Create indices of several folds
    D.make_folds(n_folds)     # Makes indices available inside class

    predictions = []
    actual_vals = []

    # for _ in range(n_folds):     # Run all folds 
    for _ in range(1):     # Run all folds
        D.next_fold()
        D.normalize()
        D.tensors_to_device()
        D.print_shapes()
        # D.plot_data()
        model = CNN(n_ch=n_press, out_size=5)      # Initialize new model each fold
        T = Trainer(D)
        T.setup(model, max_epochs=20, batch_size=int(len(D.xtr)/20))       # 20 minibatches
        T.train_model(model)
        # T.plot_train()
        R = Results(D, model)
        R.test_metrics()
        preds, actual = R.get_preds_actual()

        predictions.extend(preds)
        actual_vals.extend(actual)

    print(sklearn.metrics.classification_report(actual_vals, predictions))
    return actual_vals, predictions

In [2]:
from peratouch.config import path_analysis_results

Xraw, yraw = load_data(path_five_users_main)

number_presses = [20] # [1, 3, 5, 10, 20]

results_dict = {}
for n_press in number_presses:

    results_dict[str(n_press)] = run_n_presses(Xraw, yraw, n_press)

    np.savez(str(path_analysis_results / "number_presses.npz"), **results_dict)



-- New Fold --
Train, test and validation arrays normalized to:
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.], [0.9996 1.0002 1.0005 1.0008 1.0007 1.0004 0.9996 0.9997 0.9996 0.9997
 1.     1.0006 1.0014 1.0005 1.0006 0.9999 0.9997 1.0001 0.9996 0.9998], [0.9984 0.9985 0.9989 0.9993 0.9977 0.9964 0.998  0.998  0.9973 0.9984
 0.9988 0.9983 1.0001 1.     0.9989 0.999  0.9983 0.999  0.9994 0.9988]
Using Device:  cpu , dtype:  torch.float32

Raw data shape:  (106325, 20, 32) 
Labels shape:  (106325,) 
Shape of test set: (21265, 20, 32) 
Shape of train set: (72301, 20, 32) 
Shape of validation set: (12759, 20, 32) 
Unique labels:  [0 1 2 3 4] 
Fraction of test labels:  [0.23, 0.18, 0.21, 0.21, 0.17] 
Fraction of validation labels:  [0.23, 0.18, 0.2, 0.21, 0.16] 
Fraction of train labels:  [0.22, 0.19, 0.21, 0.21, 0.17] 
dtype of inputs:  torch.float32

 Start of training model:

End of epoch 1: loss_tr=0.819, loss_val=0.823, train=68.5%, val=68.5%
End of epoch 3: loss_tr=0

In [3]:
stored_results = np.load(str(path_analysis_results / "number_presses.npz"))

for key in stored_results:
    print(key, ":\n", sklearn.metrics.classification_report(*results_dict[key]))

20 :
               precision    recall  f1-score   support

           0       0.76      0.72      0.74      4838
           1       0.90      0.79      0.84      3930
           2       0.81      0.84      0.82      4524
           3       0.78      0.84      0.81      4398
           4       0.65      0.68      0.66      3575

    accuracy                           0.78     21265
   macro avg       0.78      0.78      0.78     21265
weighted avg       0.78      0.78      0.78     21265

